# Transformers AutoClass 

### Tokenizer, Model Loading
- Huggingface 모델 허브에서 제공하는 처리 모델을 다운받아 로딩한다.
- 다운로드된 모델은 `사용자 home 디렉토리\.cache\huggingface` 에 저장된다.
- 미리 학습된 언어 모델을 다운받아 사용할 때는 그 언어모델이 사용한 tokenizer를 같이 받아서 사용한다.

### [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto)
- Huggingface 에서 제공하는 다양한 모델들은 손쉽게 불러오고 사용할 수 있도록 설계된 유틸리티 클래스들을 말한다.
- 미리 학습된 특정 모델의 이름(모델 허브상에서 제공되는 이름)이나 저장된 local 경로를 제공하면 해당 모델에 맞는 적절한 클래스와 구성 요소를 자동으로 로드한다.
- 사용자는 모델을 사용하기 위한 정확한 클래스를 몰라도 쉽게 다양한 종류의 모델을 사용할 수있다.

#### 주요 Auto Class
- 기본 모델 Loading
    1. **AutoModel**
       - 주어진 모델 이름에 맞는 사전 학습된 모델을 자동으로 로드한다.
       - 예: `AutoModel.from_pretrained("bert-base-uncased")`: BERT 모델을 로드한다.
    2. **AutoTokenizer**
       - 해당 모델에 적합한 토크나이저를 자동으로 로드한다.
       - 예: `AutoTokenizer.from_pretrained("bert-base-uncased")`: BERT 모델에 맞는 토크나이저를 로드한다.
    3. **AutoConfig**
       - 모델의 설정(config)을 자동으로 로드한다. 모델 설정에는 모델의 하이퍼파라미터와 모델 구조 정보가 포함된다. 이 설정을 이용해 모델을 생성할 수있다.
       - 예: `AutoConfig.from_pretrained("bert-base-uncased")`
- Task 처리 모델 Loading
    - Pretrained backbone 모델에 각 task 에 맞는 estimator layer를 추가한 모델을 생성해 제공한다.
    - 주요 모델들
        1. **AutoModelForSequenceClassification**
           - 시퀀스(Text) 분류 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")`
        2. **AutoModelForQuestionAnswering**
           - 질문-응답 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")`
        3. **AutoModelForTokenClassification**
           - 토큰 분류 작업(예: 개체명 인식)을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForTokenClassification.from_pretrained("bert-base-uncased")`

In [9]:
from transformers import AutoModel, AutoConfig, AutoTokenizer

model_id = "bert-base-uncased"
# model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
print(type(tokenizer))

model = AutoModel.from_pretrained(model_id)
print(type(model))

config = AutoConfig.from_pretrained(model_id)
print(type(config))

<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
<class 'transformers.models.bert.modeling_bert.BertModel'>
<class 'transformers.models.bert.configuration_bert.BertConfig'>


In [ ]:
# config.json - 모델 아키텍쳐 관련 메타정보파일
# config
model2 = AutoModel.from_config(config) # 학습이 안된 모델.
type(model2)

transformers.models.bert.modeling_bert.BertModel

In [ ]:
from transformers import BertModel, GPT2Model
b_model = BertModel.from_pretrained(model_id)
b_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
######################
# raw-text ==> (Tokenizer) ==> token ids ==> (model) ==> 결과

In [27]:
# 토크나이저
raw_text = "I am a boy."
# 토큰화
token = tokenizer(
    raw_text,
    return_tensors="pt"  # 토큰화 결과 타입 지정. default: list, pt: pytorch, tf: tensorflow, np-numpy
    
)
token

{'input_ids': tensor([[ 101, 1045, 2572, 1037, 2879, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [28]:
# token(결과)의 key 들 조회
dict(token).keys()

# 'input_ids': 입력 토큰 id값.
# 'token_type_ids': 입력이 문장 쌍 일 때 문장을 구분하는 id. 0: 첫번째 문장, 1: 두번째 문장. 
#                   각 토큰이 어느 문장에 속했는지 토큰별로 지정됨.
#                   문장쌍: QA(질문-지문)
# 'attention_mask': 실제 문장의 토큰과 Padding을 구분한다. 토큰 위치별로 지정. 0: padding, 1: 실제 문장 토큰

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [29]:
type(token['input_ids']), type(token['token_type_ids']), type(token['attention_mask'])

(torch.Tensor, torch.Tensor, torch.Tensor)

In [30]:
######################
# token 결과 -> model

# keyword 가변인자에 맞춰서 name=value 형태로 입력. input_ids=[.....], 
# model에 맞춤 타입으로 입력. (pytorch 모델: torch.tensor)
context_vector = model(**token) # BertModel -> feature extract

In [ ]:
context_vector.last_hidden_state.shape
# 모든 입력토큰들의 hidden state들.
# [batch: 1, seq_len: 7, hidden_size(embedding vector):768]

torch.Size([1, 7, 768])

In [ ]:
# 입력 문장(문서)에 대한 context vector - 문장(문서)의 특성값.
## Bert는 last_hidden_state에서 첫번째 토큰([CLS]의 값을 가공(특성추출)해서 context vector로 사용.
context_vector.pooler_output.shape

torch.Size([1, 768])

## kcbert
- BERT 모델을 한글 텍스트로 학습 시킨 Pretrained model.
    - BERT는 Transformer의 Encoder 부분을 이용해 구현된 언어모델
    - https://arxiv.org/abs/1810.04805 
- https://huggingface.co/beomi/kcbert-base

### 토크나이저, 모델 로드

In [4]:
from transformers import AutoModel, AutoTokenizer

model_id = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)
type(tokenizer), type(model)

(transformers.models.bert.tokenization_bert_fast.BertTokenizerFast,
 transformers.models.bert.modeling_bert.BertModel)

In [ ]:
# 단일 문서 입력: padding-pad추가, truncation-잘라내기 (토큰수 맞추는 작업) 신경쓸 필요없다.
tokens = tokenizer(
    "나는 어제 친구와 밥을 먹었다.",
    return_tensors="pt"
)
# tokenizer(토큰화할 문장): 모델에 입력할 수있는 형태로 토큰결과를 반환.
tokens

{'input_ids': tensor([[    2,  8616,  9909,  9025,  4196,  4578,  4027, 21452,  4020,    17,
             3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [17]:
tokenizer.encode("나는 어제 친구와 밥을 먹었다.") # input_id만 출력
tokenizer.decode([2, 8616, 9909, 9025, 4196, 23905, 21452, 4020, 17, 3]) # token들 -> 문장
tokenizer.decode(
    [2, 8616, 9909, 9025, 4196, 23905, 21452, 4020, 17, 3],
    skip_special_tokens=True # 스페셜 토큰은 제거
)

'나는 어제 친구와 밥을 먹었다.'

In [22]:
# 개별 토큰 관련 조회
tokenizer.convert_ids_to_tokens(8616) # 토큰 ID -> 토큰 문자열
tokenizer.convert_ids_to_tokens([2, 8616, 9909])

tokenizer.convert_tokens_to_ids("어제")
tokenizer.convert_tokens_to_ids(["어제", "너는", "밖"])

[9909, 9039, 1482]

In [65]:
# 토크나이저 전체 어휘수
print(tokenizer.vocab_size)
# tokenizer.get_vocab() # 어휘사전 조회

30000


In [ ]:
###########################
# 여러 문서 입력
###########################

In [23]:
sentences = [
    "안녕",
    "Hugging Face는 인공지능(AI)과 자연어 처리(NLP) 분야에서 혁신적인 도구와 모델을 제공하는 AI 스타트업이다.",
    "2016년에 설립된 이 회사는 주로 오픈소스 라이브러리와 사전 학습된 NLP 모델을 제공을 제공한다."
]

In [24]:
sent_tokens = tokenizer(
    sentences
    # return_tensors="pt"
)

In [32]:
print(len(sent_tokens['input_ids']), sent_tokens['input_ids'])
print(len(sent_tokens['attention_mask']))
print(len(sent_tokens['token_type_ids']))

3 [[2, 19017, 3], [2, 41, 4223, 4403, 4403, 18940, 39, 4243, 4773, 4226, 4008, 14583, 25061, 11, 22502, 12, 321, 10459, 4071, 9810, 11, 47, 4450, 4579, 12, 16029, 7971, 13064, 8097, 867, 4228, 4196, 16505, 4027, 13248, 7966, 22502, 12296, 4104, 22192, 4020, 17, 3], [2, 26182, 4113, 20684, 4130, 2451, 22088, 20002, 15999, 4266, 4103, 17564, 4408, 4053, 4038, 4196, 11202, 20323, 4130, 47, 4450, 4579, 16505, 4027, 13248, 4027, 13248, 8008, 17, 3]]
3
3


In [ ]:
# 토큰수
for ids in sent_tokens['input_ids']:
    print(len(ids))

3
43
30


In [ ]:
# max_length - 최대길이(토큰수)를 지정. 

# truncation - 길이를 맞추기 위해서 토큰의 일부를 잘라내는 것.
##           - True("longest"): 배치중에서 가장 긴 문서에 맞춘다. max_length 설정이 된 경우에는 max_length에 맞춘다.
##           - False("do_not_truncate"): 기본값. Truncation을 안함.

# padding - 길이를 맞추기 위해서 [PAD] 토큰으로 채우는 것.
##        - True("longest"): 배치중에 가장 긴 문서에 맞춘다.
##        - "max_length": max_length 설정에 맞춘다. (max_length가 없으면 model의 default max_length에 맞춘다.)
##        - False("do_not_pad"): 기본값. Padding 처리 하지 안는다.

# 순서: truncation -> padding

In [ ]:
sent_tokens2 = tokenizer(
    sentences,
    max_length=20,
    padding=True,  
    truncation=True
)

In [61]:
for sent in sent_tokens2['input_ids']:
    print(len(sent))#, sent)

43
43
43


In [40]:
sent_tokens2['attention_mask'][0]

[1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

### BERT 모델을 이용해 context vector 추출
#### Model 추론결과
- **last_hidden_state**
    - 모든 token들에 대한 feature
    - 출력이 **many**인 작업에 사용한다.
- **pooler_output**
    - 입력 문장, 텍스트에 대한 context vector 이다.
    - 이 값은 **문장을 입력받아 처리하는 task**(ex: 문서분류-감정분석,문장카테고리분류, 문장유사도 분석)의 입력으로 사용한다.

In [76]:
tokens = tokenizer(
    sentences,
    max_length=20,
    padding=True,  
    truncation=True,
    return_tensors="pt"
)

In [77]:
result = model(**tokens)

In [ ]:
result.last_hidden_state.shape
# [3:batch-문서개수, 20:seq_len(토큰수), 768]

torch.Size([3, 20, 768])

In [ ]:
result.pooler_output.shape
# [3, 768]

torch.Size([3, 768])

In [81]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30000, 768, padding_idx=0)
    (position_embeddings): Embedding(300, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False